In [ ]:
# Installer les bibliothèques
#pip install jupyterlab pandas requests sqlalchemy psycopg2-binary geopandas

import requests
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
from shapely.geometry import Point
import io

# --- CONSTANTES ---
# URL de l'API Opendatasoft
# Nous limitons à 10000 enregistrements pour cet exemple. Retirez le paramètre &rows=10000 pour tout récupérer.
#API_URL = "https://public.opendatasoft.com/explore/assets/accidents-corporels-de-la-circulation-millesime/"



# --- CONFIGURATION ---
# API
API_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records"

# Nombre d'enregistrements à récupérer par appel (l'API est limitée à 100)
BATCH_SIZE = 100

# Base de données (identifiants de votre docker-compose.yml)
DB_USER = "user_securite_routiere"
DB_PASSWORD = "password123"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "securite_routiere_db"
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Crée le moteur de connexion à la base de données
engine = create_engine(DATABASE_URL)

print("Configuration chargée et moteur de base de données créé.")



Configuration chargée et moteur de base de données créé.


In [ ]:
import pandas as pd
import requests
import io

# URL corrigée avec "corporels" au lieu de "corpels"
url_csv = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

print("Lancement du téléchargement avec l'URL corrigée. Veuillez patienter...")

try:
    # 1. Effectuer la requête pour obtenir les données
    response = requests.get(url_csv)
    # Lève une exception (comme 404 Not Found ou 500 Server Error) si la requête échoue
    response.raise_for_status()

    # 2. Lire le contenu du CSV
    csv_data = io.StringIO(response.text)

    # 3. Charger les données dans un DataFrame
    df_complet = pd.read_csv(csv_data, sep=';')

    print("Téléchargement et chargement dans le DataFrame terminés avec succès !")
    print("-" * 60)
    
    # 4. Afficher le nombre de lignes et de colonnes
    nombre_lignes = df_complet.shape[0]
    nombre_colonnes = df_complet.shape[1]
    
    print(f"Le DataFrame final contient {nombre_lignes:,} lignes et {nombre_colonnes} colonnes.")
    print("-" * 60)

    # Renommer le DataFrame en 'df' pour la suite du projet, c'est plus simple
    df = df_complet.copy()
    
    print("Le DataFrame est maintenant stocké dans la variable 'df'.")
    display(df.head())


except requests.exceptions.RequestException as e:
    print(f"Une erreur de connexion est survenue : {e}")
except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")

Lancement du téléchargement avec l'URL corrigée. Veuillez patienter...


/var/folders/4_/cb0dgr6d07d_36m06pf2rqjr0000gn/T/ipykernel_1102/114913899.py:20: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_complet = pd.read_csv(csv_data, sep=';')


Téléchargement et chargement dans le DataFrame terminés avec succès !
------------------------------------------------------------
Le DataFrame final contient 475,911 lignes et 69 colonnes.
------------------------------------------------------------
Le DataFrame est maintenant stocké dans la variable 'df'.


,Identifiant de l'accident,Date et heure,Commune,Année,Mois,Jour,Heure minute,Lumière,Localisation,Intersection,...,year_georef,Nom Officiel Commune,Code Officiel Département,Nom Officiel Département,Code Officiel EPCI,Nom Officiel EPCI,Code Officiel Région,Nom Officiel Région,Nom Officiel Commune / Arrondissement Municipal,Code Officiel Commune
0,201700033619,2017-05-19T17:40:00+02:00,Perpignan,2017,5,19,17:40,Plein jour,Hors agglomération,6,...,2017,Perpignan,66.0,Pyrénées-Orientales,200027183.0,CU Perpignan Méditerranée Métropole,76.0,Occitanie,Perpignan,66136.0
1,201700048262,2017-12-25T17:55:00+01:00,Saint-denis,2017,12,25,17:55,Crépuscule ou aube,Hors agglomération,1,...,2017,Saint-Denis,93.0,Seine-Saint-Denis,200054781.0,Métropole du Grand Paris,11.0,Île-de-France,Saint-Denis,93066.0
2,201700048288,2017-01-01T17:25:00+01:00,Gennevilliers,2017,1,1,17:25,Nuit sans éclairage public,Hors agglomération,1,...,2017,Gennevilliers,92.0,Hauts-de-Seine,200054781.0,Métropole du Grand Paris,11.0,Île-de-France,Gennevilliers,92036.0
3,201700048486,2017-05-04T00:04:00+02:00,Fontenay-le-fleury,2017,5,4,00:04,Nuit avec éclairage public non allumé,Hors agglomération,1,...,2017,Fontenay-le-Fleury,78.0,Yvelines,247800584.0,CA Versailles Grand Parc (C.A.V.G.P.),11.0,Île-de-France,Fontenay-le-Fleury,78242.0
4,201700048639,2017-07-11T08:10:00+02:00,Velizy-villacoublay,2017,7,11,08:10,Plein jour,Hors agglomération,1,...,2017,Vélizy-Villacoublay,78.0,Yvelines,247800584.0,CA Versailles Grand Parc (C.A.V.G.P.),11.0,Île-de-France,Vélizy-Villacoublay,78640.0
